In [68]:
import telebot
from decouple import config
import time
import telebot, csv
from datetime import datetime
from telebot.types import LabeledPrice

In [69]:
#TOKEN de conexão do bot telegram

token = '6309691816:AAH7w-F3Zd2iMj2WneKZRhccMP6d_UIqJaE'
bot = telebot.TeleBot(token)

#TOKEN de conexão com o provedor de pagamentos

token_provedor = '350862534:LIVE:YTI0NDMyNzQ5YWNj'

In [ ]:
#Mensagem do bot em execução

print("BOT inicializando.")

In [70]:
#Criar o preço do meu produto

preco = [
    LabeledPrice(label='E-book livros para mais conhecimento', amount=1000)
]

In [71]:
# SALVAR dados da conversa com o chatbot em aquivos csv

def salvar (arquivo, conversa: list):
    with open(arquivo, 'a') as chat:
        e  = csv.writer(chat)
        e.writerow(conversa)

In [72]:
# PARTE1: Iniciar a conversa com o BOT

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, "fala, tudo bem com você ?", timeout=120)

# Função de compra

@bot.message_handler(commands=['comprar'])
def compra(message):
    bot.send_invoice(
        message.from_user.id,
        title='Livro',
        description='Livros pra ler em pdf para todos',
        provider_token=token_provedor,
        currency='BRL',
        photo_url='https://storage.googleapis.com/store.buzzeditora.com.br/wp-content/uploads/2019/12/d593c6f0-capa_a_arte_da_guerra.jpg',
        photo_size=512,
        photo_width=512,
        photo_height=512,
        is_flexible=False,
        prices=preco,
        invoice_payload="PAYLOAD"
    )

#Função de pré-checkout. Verificar dos dados do cartão
@bot.pre_checkout_query_handler(func=lambda query:True)
def verificar_cartao(pre_checkout_query):
    bot.answer_callback_query(
        pre_checkout_query.id,ok=True, error_message="Houve um erro na compra"
    )


#Função para validação de pagamento

@bot.message_handler(content_types=['successful_payment'])
def pgto_confirmado(message):
    salvar('download.csv', [message.chat.id, message.from_user.first_name, message.text, datetime.now().strftime('%d/%m/%Y %H:%M:%S')])
    doc = open("dadoteca.pdf", "rb")
    bot.send_message(message.chat.id, 'Show! pagamento confirmado! Partiu download. Já ja você receberar aqui mesmo no chat!')
    bot.send_document(message.chat.id, doc, timeout=180)
    time.sleep(2)
    bot.send_message(message.chat.id, "Obrigado pela a confiança! Querendo reiniciar a nossa converssa, basta digitar iniciar : )")




@bot.message_handler(regexp='iniciar')
def iniciar(message):
    salvar('iniciar.csv', [message.chat.id, message.from_user.first_name, message.text, datetime.now().strftime('%d/%m/%Y %H:%M:%S')])
    bot.send_message(message.chat.id, "fala, tudo bem com você ?", timeout=120)

@bot.message_handler(regexp=r'tudo|td')
def pergunta(message):
    salvar('pergunta.csv', [message.chat.id, message.from_user.first_name, message.text, datetime.now().strftime('%d/%m/%Y %H:%M:%S')])
    time.sleep(5)
    bot.send_message(message.chat.id, "Que bom bora fazer a compra do livro pdf?\n \n Apos a compro voce vai ler um dos melhores livros da sua vida,\n \n Para efetuar o pagamento clique em /comprar para efetuar o pagamendo e receber o arquivo pelo valor de 10 reais.\n \n se não escreva tchau : )", timeout=120)   


#PARTE2: Download ou Playlist

'''@bot.message_handler(regexp='bora')
def download(message):
    salvar('download.csv', [message.chat.id, message.from_user.first_name, message.text, datetime.now().strftime('%d/%m/%Y %H:%M:%S')])
    doc = open("dadoteca.pdf", "rb")
    bot.send_message(message.chat.id, "Aqui esta seu livro")
    time.sleep(2)
    bot.send_document(message.chat.id, doc, timeout=120)
    time.sleep(4)
    bot.send_message(message.chat.id, "Muito obragado, aproveite o livro. Gostaria de começa denovo ? digite iniciar")
    time.sleep(2)
    bot.send_message(message.chat.id, "Valeu e ate mais")
'''

@bot.message_handler(regexp='depois')
def convencimento(message):
    salvar('convencimento.csv', [message.chat.id, message.from_user.first_name, message.text, datetime.now().strftime('%d/%m/%Y %H:%M:%S')])
    time.sleep(6)
    bot.send_message(message.chat.id, "E serio você não vai quero, vou de dar mais uma oportulidade bora baixa, E so digitar (bora) e você tera o aquivo, do contario e so digitar (tchau) e até mais.")

@bot.message_handler(regexp='tchau')
def tchau(message):
    time.sleep(2)
    bot.send_message(message.chat.id, "Teimosão, hein!")
    time.sleep(4)
    bot.send_message(message.chat.id, "Se você gostaria de ver mais so livros olhe o video do melhor profeco de bot do Youtube, no link https://www.youtube.com/watch?v=KZCMFAc3UAM")
    time.sleep(3)
    bot.send_message(message.chat.id, "Tchau, amigo até mais. Caso mude de ideia, basta digitar iniciar o papo e baixar o aquivo.")

bot.polling() #sondagem, verificar se há mensagens